In [27]:
game_date = "1996-08-17"
get_url(game_date)

MATCH DETAILS
-------------


game_date,opposition,venue,score,competition
1996-08-17,Southend United,A,1-1,League Division 1



SUBSTITUTES
-----------


season,game_no,game_date,player_name,shirt_no,role,min_on,min_off
1996/97,1,1996-08-17,Johnny Morrissey,7,starter,NaN,NaN
1996/97,1,1996-08-17,Pat Nevin,12,sub,NaN,NaN
1996/97,1,1996-08-17,John Aldridge,8,starter,NaN,NaN
1996/97,1,1996-08-17,Ian Moore,14,sub,NaN,NaN
1996/97,1,1996-08-17,Kenny Irons,10,starter,NaN,NaN
1996/97,1,1996-08-17,Ged Brannan,13,sub,NaN,NaN



YELLOW CARDS
------------


...


RED CARDS
---------


game_date,player_name,red_cards,min_so



GOAL MINS


game_date,player_name,goal_min,penalty



LIBRARY URL:
------------
https://www.britishnewspaperarchive.co.uk/search/results/1996-08-17/1996-08-22?basicsearch=%2btranmere%20%2bSouthend&freesearch=tranmere%20Southend&place=liverpool%2c%20lancashire%2c%20england&retrievecountrycounts=false&mostspecificlocation=liverpool%2c%20lancashire%2c%20england&sortorder=dayearly


In [1]:
import pandas as pd
import pyperclip
from IPython.display import display, HTML

def get_url(date, area = "liv"):
    results = pd.read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", parse_dates = ["game_date"], usecols = ["game_date", "opposition", "venue", "competition", "score"])
    
    print("MATCH DETAILS")
    print(len("MATCH DETAILS") * "-")
    results = results[results.game_date == date]

    display(HTML(results.to_html(index = False)))

    date_1 = results[results.game_date == date].game_date.astype(str).values[0]
    date_2 = results[results.game_date == date].game_date + pd.Timedelta(days=5)
    date_2 = date_2.astype(str).values[0]

    opp = results[results.game_date == date].opposition.str.split(' ').str[0].values[0]
    
    if area == "liv":
        url = f"https://www.britishnewspaperarchive.co.uk/search/results/{date_1}/{date_2}?basicsearch=%2btranmere%20%2b{opp}&freesearch=tranmere%20{opp}&place=liverpool%2c%20lancashire%2c%20england&retrievecountrycounts=false&mostspecificlocation=liverpool%2c%20lancashire%2c%20england&sortorder=dayearly"
    else:
        url = f"https://www.britishnewspaperarchive.co.uk/search/results/{date_1}/{date_2}?basicsearch=%2btranmere%20%2b{opp}&freesearch=tranmere%20{opp}&&retrievecountrycounts=false&sortorder=dayearly"

    sub_mins = pd.read_csv("./manual/subs_missing_mins.csv", parse_dates = ["game_date"])
    sub_mins = sub_mins[sub_mins.game_date == date]
    
    print("\nSUBSTITUTES")
    print(len("SUBSTITUTES") * "-")
    if len(sub_mins) == 0:
        print("No subs.\n")
    else:
        display(HTML(sub_mins.to_html(index = False)))

    yc = pd.read_csv("./manual/cards_yellow.csv", parse_dates = ["game_date"])
    yc = yc[yc.game_date == date]
    print("\nYELLOW CARDS")
    print(len("Yellow cards") * "-")
    display(HTML(yc.to_html(index = False)))

    rc = pd.read_csv("./manual/cards_red.csv", parse_dates = ["game_date"])
    rc = rc[rc.game_date == date]
    print("\nRED CARDS")
    print(len("Red cards") * "-")
    display(HTML(rc.to_html(index = False)))

    goal_mins = pd.read_csv("./manual/goal_mins.csv", parse_dates = ["game_date"])
    goal_mins = goal_mins[goal_mins.game_date == date]
    print("\nGOAL MINS")

    display(HTML(goal_mins.to_html(index = False)))

    print("\nLIBRARY URL:")
    print(len("Library URL:") * "-")
    print(url)

    pyperclip.copy(date)

In [28]:
def save_files():
    sub_mins = pd.read_csv("./manual/subs_missing_mins.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date"])
    sub_mins.to_csv("./manual/subs_missing_mins.csv", index = False)

    yc = pd.read_csv("./manual/cards_yellow.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date"])
    yc.to_csv("./manual/cards_yellow.csv", index = False)

    rc = pd.read_csv("./manual/cards_red.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date", "min_so"])

    goal_mins = pd.read_csv("./manual/goal_mins.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date", "goal_min"])

save_files()